## 一、维数诅咒  
维数越高，样本之间的距离越远，这会导致数据集有非常稀疏的风险

## 二、降维的主要方法

**2.1 投影**  
并不总是最好的方法，例如：将瑞士卷数据集投影到二维

**2.2 流形学习**  
通过对训练集所处的流形进行建模的许多降维算法，称作流形学习

## 三、PCA  
首先识别出位于数据最近的超平面，然后将数据投影到上面

**3.1 保留方差**  
方法一：选择能够保留最大方差的轴  
方法二：选择能够使原始数据集和它在轴上投影之间均方距离最小的轴

**3.2 主成分**  
$X=U\cdot∑\cdot V^{T}$  
$X_{m\times n}$，$U_{m\times m}$，$∑_{m\times n}$，$V^{T}_{n\times n}$  
$X$：训练集矩阵  
$V^{T}$：第i列即为第i个主成分

In [1]:
# 构建三维数据集
import numpy as np

np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles) / 2 + noise * np.random.rand(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [2]:
# 用SVD获得训练集的所有主成分
X_centered = X - X.mean(axis=0)
U, s, V = np.linalg.svd(X_centered)
c1 = V.T[:, 0]
c2 = V.T[:, 1]

In [3]:
U.shape, s.shape, V.shape

((60, 60), (3,), (3, 3))

In [4]:
S = np.zeros((60, 3), dtype=complex) # dtype=complex：复数类型
S[:3, :3] = np.diag(s)

**3.3 向下投影至d维**  
$X_{d-proj}=X\cdot W_{d}$  
$W_{d}$：取出$V^{T}$中前$d$列向量组成的矩阵

In [5]:
w2 = V.T[:, :2]
X2D = X_centered.dot(w2)

**3.3 用Scikit-Learn完成**

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

**3.4 方差解释率**

In [7]:
print(pca.explained_variance_ratio_)

[0.84788438 0.14210985]


**3.5 选择恰当的维数**

In [8]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
d

2

In [9]:
# 上面找到保留95%方差的维数，然后再设置n_components=d执行PCA降维
# 另一种更好的方法：设置n_components为0.0到1.0之间的数，表示保留的方差解释率
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

**3.6 PCA用于压缩**

In [10]:
# 获取minst数据集
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original')
X_mnist = mnist["data"]

In [11]:
pca = PCA(n_components = 154)
X_mnist_reduced = pca.fit_transform(X_mnist)
X_mnist_recovered = pca.inverse_transform(X_mnist_reduced) # 解压缩

PCA逆变换，解压缩回原来的维数：  
$X_{recovered}=X_{d-proj}\cdot W_{d}^{T}$ 

**3.7 增量PCA**

In [12]:
# 之前执行PCA时，会一次性将整个数据集写入内存，增量PCA解决这个问题
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist, n_batches):
    inc_pca.partial_fit(X_batch)

X_mnist_reduced = inc_pca.transform(X_mnist)

In [13]:
# 或者，使用Numpy的memmap类，允许你操作存储在磁盘上的二进制文件中的大型数组
# 由于增量PCA类在任何给定时间都用数组的一小部分，所以内存在节制下，可以调用fit()方法
import os

path = os.getcwd()
filename = os.path.join(path, "my_mnist.data").replace('\\', '/')
m, n = X_mnist.shape
X_mm = np.memmap(filename, dtype="float32", mode="write", shape=(m,n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

C:\Users\LeoWang\Anaconda3\lib\site-packages\sklearn\decomposition\incremental_pca.py:259: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)


IncrementalPCA(batch_size=700, copy=True, n_components=154, whiten=False)

**3.8 随机PCA**  
计算复杂度为：$O(m\times d^{2})+O(d^{3})$  
而不是先前的：$O(m\times n^{2})+O(n^{3})$

In [15]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_mnist)

## 四、核PCA  
可以执行复杂的非线性投影以降低维度  
通常可以很好地保存投影后地样本簇，或者擅长展开在扭曲流形附件地数据集

In [16]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

**4.1 选择核函数和调整超参数**  
虽然PCA是无监督学习算法，但通常它是监督学习任务地准备步骤，  
因此可以通过网格搜索寻找在任务上表现最好的核函数和超参数

In [21]:
from sklearn.datasets import make_swiss_roll

X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
y = t > 6.9

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
    ])

param_grid = [{
        "kpca__gamma":np.linspace(0.03, 0.05, 10),
        "kpca__kernel":["rbf", "sigmoid"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('kpca', KernelPCA(alpha=1.0, coef0=1, copy_X=True, degree=3, eigen_solver='auto',
     fit_inverse_transform=False, gamma=None, kernel='linear',
     kernel_params=None, max_iter=None, n_components=2, n_jobs=1,
     random_state=None, remove_zero_eig=False, tol=0)), ('log_reg', LogisticRegre...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'kpca__gamma': array([0.03   , 0.03222, 0.03444, 0.03667, 0.03889, 0.04111, 0.04333,
       0.04556, 0.04778, 0.05   ]), 'kpca__kernel': ['rbf', 'sigmoid']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [29]:
print(grid_search.best_params_)

{'kpca__gamma': 0.043333333333333335, 'kpca__kernel': 'rbf'}


另一种方法：选择核函数和超参数产生最小重构误差

In [31]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, 
                   fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [34]:
# 计算重构误差
from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage)

32.78630879576613

## 五、LLE（局部线性嵌入）  
另一种强大的非线性降维技术，是一种不依赖于投影的流形学习技术

In [35]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

LLE步骤1：对局部关系进行线性建模  
$\pmb{\widehat{W}}=\underset{\pmb{W}}{argmin}\sum_{i=1}^{m}\left \| \pmb{x}^{(i)}-\sum_{j=1}^{m}w_{i,j}\pmb{x}^{(j)} \right \|^{2}$  
$s.t.\left\{\begin{matrix}
w_{i,j}=0 &if\,\pmb{x}^{(j)}不是\pmb{x}^{(i)}的k近邻中的一个 \\ 
\sum_{j=1}^{m}w_{i,j}=1 & i=1,2,\cdots ,m
\end{matrix}\right.$

LLE步骤2：保留关系的同时降低维度  
$\widehat{\pmb{Z}}=\underset{\pmb{Z}}{argmin}\sum_{i=1}^{m}\left \| \pmb{z}^{(i)}-\sum_{j=1}^{m}\widehat{w}_{i,j}\pmb{z}^{(j)} \right \|^{2}$

## 六、其他降维技巧  
**多维缩放（MDS）**：试图保持样本间距离同时降低维数  
**Isomap**：通过将每个样本连接到最近邻来构造近邻图，然后试图保持样本之间的测地距离同时降低维度  
**t-分布随机近邻嵌入（t-SNE）**：试图让相似样本接近，不相似样本远离的同时降低维数。主要用于可视化，尤其用于可视化高维空间中的样本簇  
**线性判别分析（LDA）**：实际为一种分类算法，在训练过程中，它会学习类之间最有辨别性的轴，然后使用这些轴定义为投影数据的超平面。好处是投影会尽可能保持类之间的距离，所以在执行另一种分类算法（如SVM分类器）之前，LDA是降低维数的好方法